In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler  
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationSummaryBufferMemory
import time

# 모든 동작이 역순으로 되어 있다. 이런 방식에 익숙 하지 않다. 반드시 숙지 할 필요가 있어 보인다
# 랭스미스는 랭체인의 추척과 관련되어 있다. nico 선생님은 랭 스미스를 통하여 어떤 chain 을 통하여 llm 에 전달하고 전달받고 하는 내용을 알리고 싶어 했다. 
# map 형식으 도큐먼트 로드 방식은 어떤 데이터가 llm 에 전달되는지 확실하게 체크하기가 여간 번거로우니 그 번거로움을 해결하고자 랭 스미스를 활용한것으로 보인다.
# 따라서 본 과정은 랭스미스 없이도 동작이 되고 bp 등을 활용하여 디버그가 가능하지만 그걸 배제하고 랭스미스를 활용했다.... 
# 그러니 반드시 ai 과정의 prompt chain 을 확인하고 싶으면 랭스미스를 통하여 확인 하도록 하자....
# 맵을 생성 하는 과정에서 스킵이 안된다..... 따로 설정을 해줘야 하는데 시간 관계상 다음 기회에 하도록 하자... 이럴꺼면 메모리를 저장하는 이유가 없다.. stuff 방식이라면 모를까

llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",             # 역시 랭체인 memory 를 사용하면 cl100k_base encoding 에러가 발생한다 바꾸는것이 간단 할 것 같은데 모르겠다 
    # model="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

memory = ConversationSummaryBufferMemory(                  # 마지막 예제를 그대로 따른다
    llm=llm,
    max_token_limit=120,                                   # 120 이 정확하게 뭔지는 모르겠지만 비용이라는 것은 알겠다
    return_messages=True,                                  # 이걸 해 줘야 llm 이 알아 먹을 수 있다 안하면 걍 str 형식으로 저장하게 된다. 저장된 내용을 다시 전달 해 줘야 되지 않겠는가
)

cache_dir = LocalFileStore("./.cache/")                                      # from langchain.storage import LocalFileStore 로 불러온 함수를 이용하여 로컬 캐쉬 폴더를 지정한다                                                     

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(             # 텍스트를 나누는 스플리터 를 스플리터 메소드인 from_tiktoken_encoder로 초기화 한다(인스턴스를 만든다가 맞겠지?), 
                                                                             # 클라스는 하나가 더 있다 바로 RecursiveCharacterTextSplitter 이건 재귀적 스플리터 인것으로 보인다, 의미를 부여하여 나누게 됨으로 보다 발전적인 방법이다. 결과
    # separator="\n",                                                        # 청크를 자를 기본 구분점을 칸 띄우기로 했다(CharacterTextSplitter 일때...), RecursiveCharacterTextSplitter 일때에는 의미별로 이넘아가 생각을 하게 된다
    chunk_size=600,                                                          # 청크가 덩어리 이니 덩어리의 싸이즈그 600 이된다... 600자 인지 토큰인지 모르겠음.
    chunk_overlap=100,                                                       # 청크는 앞 뒤로 100 글짜인지 100 토큰인지가 곂친다
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")                   # 스플릿할 파일을 로드한다 클라스 형태로 되어 있고 loader 인스턴스를 생성한다

docs = loader.load_and_split(text_splitter=splitter)                         # 로드를 함과 동시에 스필릿을 해준다 메서드 형태이다 텍스트 스플리터는 from_tiktoken_encoder메서드로 인스턴스화 했기 때문에 매개변수로 입력해 주고 클라스는 loader 이다

# print(docs)  # 잘 되는지 확인 해 볼 수 있다

embeddings = OpenAIEmbeddings()                                                             # 오픈AI 의 인베딩 클래스를 인스턴스화 한다. 인베딩 할 기본 데이터 포맷으로 사용할라 함(오브젝트), 이게 AI 가 바뀌면 바뀔 수 도 있을찌도 모른다.. AI 마다 인베딩 형식이 틀린꺼 아닌가....

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)           # 어떤 모델의 인베딩을 지정하는것과 캐쉬의 디렉토리를 알려 줘서 캐쉬백 그니깐 저장될 인베딩 데이터를 오브젝트화 한다.AI 가 바뀌면 틀려질찌도 모른다

vectorstore = FAISS.from_documents(docs, cached_embeddings)                                 # 벡터 스토어를 지정한다... 이 단계에서 캐쉬 파일이 생성된다.. 매개변수로 스플릿 시킨 도큐먼트와 위에서 만든 인스턴스를 너어주고 이에 따른 로컬 캐쉬가 생성된다
              # 검색해 보니 페이스북의 벡터 스케일 방식이다. 물론 다른 벡터스케일을 사용 해도 된다
retriever = vectorstore.as_retriever()                                                      # 우리가 사용할 벡터 스토어 라는것은 알겠다.... 정확하게 알고 넘어가자.... 지금은 잘 모름. vectorstore 라는것은 저장소를 말하고 여기서 무언가를 찾기 위하여 초기화 시켜주는 개념으로 이해하고 있다.

# 어떻게 보면 여기서부텀 역순으로 추가해 나가는게 편해 보인다 nico 선생님의 독특한 방식인건지 그건 잘 모르겠으나 구분점은 확실하게 있다 

map_doc_prompt = ChatPromptTemplate.from_messages(                                          # 맵방식의 도큐먼트 로딩을 위하여 프롬프트를 하나 만들어 준다. 
    [
        (
            "system",                                                                       # 시스템은 제공되어진 텍스트{context}에서 질문과 관련된 내용을 찾아서 답하게 되었다. 만일 {context}에 내용이 없다면 '' 로 대답하게 되어 있다
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. 
            Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """
        ),
        (
            "human", 
            "{question}"
        )                                                                                   # 여기에 질문이 들어간다
    ]
)

map_doc_chain = map_doc_prompt | llm                                                        # 위의 프롬프트로 체인을 만든다. 이 프롬프트를 어케 실행하느냐 그것이 문제로다. 오로지 맵을 위해서 만든 체인이다


# 여기서부터 제일 헤깔리는 부분이다... 이런 방식은 늘 복잡하다

def map_docs(inputs):                                                                       # 맵 방식의 도큐먼트 로드를 위하여 반복 실행을 해 준다 입력을 리스트로 받는다... 그럼 도대체 input 은 어떻게 오는가 인데... "이게 바로chain" 이다
    documents = inputs["documents"]                                                         # 체인은 | 이러한 방식으로 규정 되어진다. 참으로 충격적인 방식이다. | 하나 쓴다고 해서 체인이 되다니 개 충격적이다.. a_chain = "qqq" | "www" 이렇게 해도 체인이 된다.. 파이썬에선 이런게 말이되는것이다.
    question = inputs["question"]                                                           # 후에 생성할 체인에 A|B 방식의 체인으로 이 함수가 람다로 쓰여지게 되는데 매개변수/파라미터 를 체인으로 받게 된다... 이게 가장 핵심이다... 
    
    # return "\n\n".join(map_doc_chain.invoke({"context": doc.page_content, "question": question}).content for doc in documents)
    # 현재 신규 가입자는 1분에 3회 요청이 가능하다. 리미트가 걸려있는 셈이다 이 리미트는 결재를 한다고 해서 올라가지는 않는다
    # 따라서 위와 같은 신통방통한 syntex(구문)로는 신규 가입자로써 llm 동작에 리미트가 걸리기 때문에 못쓴다.. 라기 보다는 지연시간 주는 방법을 모르겠다
    # 때문에 아래와 같은 고전적인 방식으로 20초 딜레이 시간을 준다 
    # 이 리미트는 사용양이 5달러를 넘어가면 다음 스텝의 리미트로 올라가게 되는 구조이다
    # 다른 리미트 해제 방법이 있는지는 모르겠다. (2024_0901)
    
    results = []                                                                                                    # 검색의 결과물을 담을 리스트를 생성하고
    for doc in documents:                                                                                           # 반복문을 돌려서 입력받은 도큐먼츠 즉 캐쉬를 검색한다. 스플릿된 큐먼트의 내용은 page_content, metadata 등으로 구분되어져 있다... print("context") 해 보면 알수 있을 것이다
        result = map_doc_chain.invoke({"context": doc.page_content, "question": question}).content                  # 맵 제작을 위한 체인을 실행하여 결과를 찾아낸다. find 가 아니라 도큐먼츠 안에 있는 스플릿 된 문서를  ai 에 전달하여 내용이 있는지 찾는것이다. 네가보기엔 이 스플릿 문서에 질문의 답이 있냐? 뭐 이런식
        results.append(result)                                                                                      # 없으면 '' 를 리턴하게 됨으로 건너 뛴다.. 있으면 그 값을 리스트에 넣는다
        time.sleep(20)                                                                                              # 위의 사정으로 1분간 3번뿐이 llm 을 사용하지 못한다. 때문에 20초 딜레이를 둔다 5달러가 넘어가면 리미트는 해제 되는거 같다

    results = "\n\n".join(results)                                                                                  # 그 결과(리스트를 두줄 띄우기로 합치고) 리터럴 문자열 즉 str 이외에는 변수에 들어갈 수 없다...
    print(results)                                                                                                  # 결국에 질문에 대한 답이 있는 내용들만 리스트 들어가고 합쳐지게 된다. 리턴값을 프린트 해보면 확인이 가능하다.
    return results                                                                                                  # 체인을 위한 리턴을 해 준다 결과 값만 나오게 된다                                                                                           
   
map_chain = {                                                                                                       # 가장 이해가 어려운 부분 일 수 있다. 문서를 전달하여 map 을 만드는 체인이다. A | B 형식으로 되어 있음이다. B 에 필요한 딕셔너리를 제공해 준다. 
    "documents": retriever,                                                                                         # 도큐먼츠는 스필릿해서 만든 캐쉬 데이터 이고 이건 벡터스토어 형식으로 변수로 만들어 졌다 이게 도큐먼츠 이다 
    "question": RunnablePassthrough(),                                                                              # 질문은 입력에서 받아온다
} | RunnableLambda(map_docs)                                                                                        # 프롬프트 형태의 데이터가 아니기 때문에 RunnablePassthrough 와 마찬가지로 RunnableLambda 를 활용하여 함수를 실행해 준다

final_prompt = ChatPromptTemplate.from_messages(                                                                    # 최종 프롬프트를 생성해 준다. 시스템 또한 "추출해 낸" 이 들어가 있다 이것을 바탕으로 마지막 답을 하라고 지시해 준다
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """
        ),
        MessagesPlaceholder(variable_name="history"),               # 질문 하기전에 메세지 플레이스 홀더를 사용하여 지난 질문의 히스토리를 참조한다. 순서상 다른데 있으면 엉뚱한 대답이 나올 수 있다
        (
            "human", 
            "{question}"
        )                                                    
    ]
) 

def load_memory(_):                                                                 # history 라는 키의 메모리를 불러와서 리턴하는 함수..... 형태가 괴상하다. 매개변수가 있어야 동작한다고 하니 더미(?) 인 _ 를 너어주는것 같다                                                                               
    return memory.load_memory_variables({})["history"]                                                               


# 여기가 최종 체인이다. 첫번째 체인에서는 제작한 맵이 들어가고 히스토리는 메모리에서 호출 한다. 로드 메모리에서 인풋을 줘야 한다 여기서 엄청 헤멧다. RunnablePassthrough.assign(history = load_memory) 이건 프롬프트 형태이다.. 여기서는 사용 할 수 없다
# 원칙적으로는 람다를 사용 해야 한다. 마우스를 올려놔 보면 Runnable 타입이 와야 한다고 나와 있다.  그렇기 때문에 위에서처럼 러너블 람다로 실행해 준다. 여기까지는 이해가 간다
# 신기하게도  "history" 의 딕셔너리는 load_memory 만 해 줘도 동작을 한다. 이유는 모른다. 위에 있는 함수는 input 이라는 입력이 있어서인지 전현 모르겠다. 위의 map_docs 는 RunnableLambda없이 사용이 안된다. 단순히 데이터 형식이 맞고 안맞고이기 때문인거 같기도 하다.
chain = {"context": map_chain, "history": load_memory  ,"question": RunnablePassthrough()} | final_prompt | llm         # 맵을 만들기 위한 전달용 프롬프트를 실행 후 맵이 만들어지면 마지막 프롬프트에게 내용을 전달 후 llm 에 프롬프트를 전달하는 체인 이다


# 메모리를 저장 해야 중복 동작을 방지 할 수 있다.... 그러나 완벽하지 않다. map_docs 단계가 동작하기 때문이다.... 조금만 더 하면 그 부분을 건너 뛰어서 빠르게 결과를 볼 수 있을것 같으나 너무 졸려서 다음 기회로 미룬다.....
def invoke_chain(question):                                                                      # 메모리에 결과값을 전달하기 위하여 함수를 사용했다 매개변수는 아마 str 이겠다
    result = chain.invoke(question)                                                # 아우풋을 만들어 내서 result 에 담고
    memory.save_context(                                                                         # 메모리에 저장한다
        {"input": question},                                                                     # 딕셔너리 형태로 2개의 콘텍스트를 너을 수 있다 인풋에는 매개변수 입력값을
        {"output": result.content}                                                               # 아웃풋에는 결과값을 넣는데 맞는 형식을 너어줘야 한다 걍 너으면 에러남           
    )
    print(result.content)   ## 보여준다


invoke_chain("Is Aaronson guilty?")                                                              # 중복 메모리 참조가 되는지 확인을 위하여 일단 질문
                                                                                                 # 오류의 공론화를 위해 일단 오류창을 그대로 커밋한다


